### A sequential workflow to load tif images in separate channels and view using napari QT

In [1]:
# load qt5 gui magic
%gui qt5

import warnings
warnings.filterwarnings('ignore')
# import image read from dask + import napari
from dask_image.imread import imread
from naparimovie import Movie
import napari

In [2]:
# for viewing of dask processes while using the napari gui
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:53110 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 548.31 GB


In [3]:
# direct path to tif images, separate into two batches via channel search (e.g. ch0)
stacks0 = imread('C:\\Users\\Viz2\\python_anaconda3\\UCB ABC\\demo_napari\\GPU\\*ch0*.tif')
stacks1 = imread('C:\\Users\\Viz2\\python_anaconda3\\UCB ABC\\demo_napari\\GPU\\*ch1*.tif')

In [5]:
# view nestled images in qt
viewer = napari.Viewer(ndisplay=3)  # stack tif images into 3D format
layer1 = viewer.add_image(stacks0, scale=(5,1,1), name="first channel", is_pyramid=False,
                         colormap='cyan', opacity=.5, blending='additive')
layer2 = viewer.add_image(stacks1, scale=(5,1,1), name="second channel", is_pyramid=False,
                         colormap='magma', opacity=.7, blending='additive')
# viewer.window._qt_window.raise_()
movie = Movie(myviewer=viewer)

In [ ]:
movie.make_gif('gif_interactive.gif')

### BELOW WAS CREATED IN 2020-02-13 FOR VIEWING ORIGINALLY 125 GB FILE OF TIFF
### CONVERT BELOW TO MOVIE AFTER EDITING ON NAPARI

-------------------------------------------------------------------------------

In [1]:
%gui qt5

In [6]:
import os
os.chdir(r'C:\Users\Viz2\python_anaconda3\UCB ABC\git_clones\LLS_Pipeline\Modules\Zarr')
print(os.listdir())

from dask_image.imread import imread
import read_zarr
import dask.array as da
from naparimovie import Movie
import napari
zarr_filepath = r'C:\Users\Viz2\python_anaconda3\UCB ABC\git_clones\LLS_Pipeline\Exercises\Zarr\test_mydata.zarr'

d = read_zarr.retrieve_zarr(zarr_filepath, 'dask')
channel = d.shape.index(min(d.shape))
blobs = da.stack(d)
viewer = napari.Viewer(ndisplay=3)
# viewer = napari.view_image(blobs.astype(float))
viewer.add_image(blobs, channel_axis=channel)

movie = Movie(myviewer=viewer)

['napari_zarr.py', 'read_zarr.py', 'write_zarr.py', '__pycache__']


In [7]:
movie.make_gif('zarr_interactive.gif')

-------------------------------------------------------------------------------------

In [9]:
import pycuda.compiler as comp
import pycuda.driver as drv
import numpy
import pycuda.autoinit

mod = comp.SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(400).astype(numpy.float32)
b = numpy.random.randn(400).astype(numpy.float32)

dest = numpy.zeros_like(a)
multiply_them(
        drv.Out(dest), drv.In(a), drv.In(b),
        block=(400,1,1))

print (dest-a*b)

ModuleNotFoundError: No module named 'pycuda'